Set path as '/scratch/ECE8803_Project'. 

In this folder, you need to upload 'sam' data, label and samples.

In [1]:
# Set Path
import os
local_disk_root=os.environ["HOME"] + '/scratch'
root = local_disk_root+'/ECE8803_Project'
os.chdir(root)

Unpack 'Prompting results.zip' file. 

We are only going to use 'Dolphin Below', 'Cat', and 'Bus'.

In [2]:
# Unpack Prompt Result (Run only once)

# from zipfile import ZipFile
# import shutil

# zipobj = ZipFile('Prompting results.zip')
# filenames = zipobj.namelist()[1:]  # all fileIds in the folder

# with ZipFile('Prompting results.zip','r') as zipObject:
#   for filename in filenames:
#      zipObject.extract(filename)

In [3]:
# Set Package
import numpy as np
import torch
import cv2
from openpyxl import Workbook
import os
import random
from IPython import get_ipython
import time
from PIL import Image
from segment_anything import sam_model_registry, SamPredictor

Load SAM Model

In [4]:
sam_checkpoint = 'sam_vit_h_4b8939.pth'
model_type = "vit_h"
device = "cuda" if torch.cuda.is_available() else 'cpu'
sam = sam_model_registry[model_type](checkpoint=sam_checkpoint)
sam.to(device=device)
predictor = SamPredictor(sam)

Set User Parameters for Rescale Analysis

In [5]:
dataset_name = "Dolphin below" # Dolphin below, Cat, Bus, Polyp, Stop Sign
max_samples = 100
num_student = 3
rescale_ratio_set = [1.25, 1.5, 1.75, 2.0]
mode_set = ['nearest', 'bilinear', 'bicubic', 'area']

Check Sample and Label exist

In [6]:
names = np.load(dataset_name+"_samples.npy", allow_pickle=True)
labels = np.load(dataset_name+"_labels.npy", allow_pickle=True)

if not os.path.exists(dataset_name):
    os.makedirs(dataset_name)

In [7]:
## Rescale the image and mask using Torch Interpolation
def RescaleImage(image, size, mode='nearest'):
    imageRawTensor = torch.from_numpy(image).permute(2, 0, 1).unsqueeze(0).float()
    imageRescaleTensor = torch.nn.functional.interpolate(imageRawTensor, size=size, mode=mode)
    imageRescale = imageRescaleTensor[0].permute(1, 2, 0).byte().numpy()
    return imageRescale

def RescaleMask(maskBool, size, mode='nearest'):
    maskRawTensor = torch.from_numpy(maskBool).unsqueeze(0).unsqueeze(0).float()
    maskRescaleTensor = torch.nn.functional.interpolate(maskRawTensor, size=size, mode=mode)
    maskRescale = maskRescaleTensor[0,0].numpy()
    maskRescaleBool = maskRescale > 0.5
    return maskRescaleBool

Run and Test the Performance

In [ ]:
## start looping through samples:
for student in np.arange(1,num_student+1):
    print("Ratio Student: ", student) 
    wb = Workbook()
    ws = wb.active
    ws['A1'] = 'Sample Number'
    ws['B1'] = 'Scores'
    ws['C1'] = 'Time Set'
    ws['D1'] = 'Time Predict'
    ws['E1'] = 'Rescale Ratio'
    
    num_files = 0
    dir_path = "Prompting results/" + dataset_name + "/st" + str(student)
    
    # Count the number of samples
    for path in os.listdir(dir_path + "/scores/"):
        # check if current path is a file
        if os.path.isfile(os.path.join(dir_path + "/scores/", path)):
            num_files += 1      

    # Test for Different Scale ratio
    idx=0
    for rescale_ratio in rescale_ratio_set:
        mode = 'area'
        c=0
        while c < num_files and c < max_samples:
            # Load Prompting result and use max score prompt
            score_points = np.load(dir_path + "/scores/" + str(c) + "score.npy", allow_pickle=True)
            if score_points.size == 0:
                ws['A' + str(c + 2)] = str(c)
                ws['B' + str(c + 2)] = str(0)  # samples name on excel
                ws['C' + str(c + 2)] = str(0)
                c += 1
                continue
            green_points = np.load(dir_path + "/points/" + str(c) + "_green.npy", allow_pickle=True)
            if green_points.size == 0:
                ws['A' + str(c + 2)] = str(c)
                ws['B' + str(c + 2)] = str(0)  # samples name on excel
                ws['C' + str(c + 2)] = str(0)
                c += 1
                continue
            green_points = green_points[np.argmax(score_points)]
            red_points = np.load(dir_path + "/points/" + str(c) + "_red.npy", allow_pickle=True)
            if red_points.size == 0:
                ws['A' + str(c + 2)] = str(c)
                ws['B' + str(c + 2)] = str(0)  # samples name on excel
                ws['C' + str(c + 2)] = str(0)
                c += 1
                continue
            red_points = red_points[np.argmax(score_points)]
            gp = []  # green points
            rp = []  # red points

            ## Load Image
            image = names[c]  # samples c
            if len(image.shape) == 2:
                image = cv2.cvtColor((np.array(((image + 1) / 2) * 255, dtype='uint8')), cv2.COLOR_GRAY2RGB)
            imshape = image.shape[0],image.shape[1]
            if np.max(image) < 2:
                image *= 255

            ## Downsample the image
            imreshape = (int(imshape[0] * rescale_ratio), int(imshape[1] * rescale_ratio))
            imageDown = RescaleImage(image, imreshape, mode)

            ## Load Label
            label = labels[c]  # GT for sample c
            label = label == 1
            mask = 0

            ## Set Downsampled Image
            time_part1i = time.time()

            predictor.set_image(imageDown)

            time_part1 = time.time() - time_part1i

            ## Concatenate Green and Red Prompts
            x,y = 0,0
            green = []
            for g in green_points:
                x = g[0] * rescale_ratio
                y = g[1] * rescale_ratio
                green.append((x, y))
            red = []
            for r in red_points:
                x = r[0] * rescale_ratio
                y = r[1] * rescale_ratio
                red.append((x, y))

            ## Predict Downsampled Image
            score = 0.
            time_part2 = 0.
            if green and red:
                input_point = np.concatenate((green, red))
                input_label = np.concatenate(([1] * len(green), [0] * len(red)))

                time_part2i = time.time()
                masks, scores, logits = predictor.predict(
                    point_coords=input_point,
                    point_labels=input_label,
                    multimask_output=True,
                )
                time_part2 = time.time() - time_part2i

                ## Upsample the mask
                maskDown = masks[0]
                mask = RescaleMask(maskDown, imshape, mode)

                ## Score calculation
                intersection = (mask & label).sum()
                union = (mask | label).sum()
                if intersection == 0:
                    score = 0
                else:
                    score = intersection / union

            ws['A' + str(idx + 2)] = c+1
            ws['B' + str(idx + 2)] = score
            ws['C' + str(idx + 2)] = time_part1
            ws['D' + str(idx + 2)] = time_part2
            ws['E' + str(idx + 2)] = rescale_ratio
            c += 1
            idx += 1
            
    wb.save(os.path.join(dataset_name, dataset_name + '_st' + str(student) + '_upscale.xlsx'))
    

## start looping through samples:
for student in np.arange(1,num_student+1):
    print("Mode Student: ", student) 
    wb = Workbook()
    ws = wb.active
    ws['A1'] = 'Sample Number'
    ws['B1'] = 'Scores'
    ws['C1'] = 'Time Set'
    ws['D1'] = 'Time Predict'
    ws['E1'] = 'Mode'
    
    num_files = 0
    dir_path = "Prompting results/" + dataset_name + "/st" + str(student)
    
    # Count the number of samples
    for path in os.listdir(dir_path + "/scores/"):
        # check if current path is a file
        if os.path.isfile(os.path.join(dir_path + "/scores/", path)):
            num_files += 1      

    # Test for Different Scale ratio
    idx=0
    for mode in mode_set:
        rescale_ratio = 2.0
        c=0
        while c < num_files and c < max_samples:
            # Load Prompting result and use max score prompt
            score_points = np.load(dir_path + "/scores/" + str(c) + "score.npy", allow_pickle=True)
            if score_points.size == 0:
                ws['A' + str(c + 2)] = str(c)
                ws['B' + str(c + 2)] = str(0)  # samples name on excel
                ws['C' + str(c + 2)] = str(0)
                c += 1
                continue
            green_points = np.load(dir_path + "/points/" + str(c) + "_green.npy", allow_pickle=True)
            if green_points.size == 0:
                ws['A' + str(c + 2)] = str(c)
                ws['B' + str(c + 2)] = str(0)  # samples name on excel
                ws['C' + str(c + 2)] = str(0)
                c += 1
                continue
            green_points = green_points[np.argmax(score_points)]
            red_points = np.load(dir_path + "/points/" + str(c) + "_red.npy", allow_pickle=True)
            if red_points.size == 0:
                ws['A' + str(c + 2)] = str(c)
                ws['B' + str(c + 2)] = str(0)  # samples name on excel
                ws['C' + str(c + 2)] = str(0)
                c += 1
                continue
            red_points = red_points[np.argmax(score_points)]
            gp = []  # green points
            rp = []  # red points

            ## Load Image
            image = names[c]  # samples c
            if len(image.shape) == 2:
                image = cv2.cvtColor((np.array(((image + 1) / 2) * 255, dtype='uint8')), cv2.COLOR_GRAY2RGB)
            imshape = image.shape[0],image.shape[1]
            if np.max(image) < 2:
                image *= 255

            ## Downsample the image
            imreshape = (int(imshape[0] * rescale_ratio), int(imshape[1] * rescale_ratio))
            imageDown = RescaleImage(image, imreshape, mode)

            ## Load Label
            label = labels[c]  # GT for sample c
            label = label == 1
            mask = 0

            ## Set Downsampled Image
            time_part1i = time.time()

            predictor.set_image(imageDown)

            time_part1 = time.time() - time_part1i

            ## Concatenate Green and Red Prompts
            x,y = 0,0
            green = []
            for g in green_points:
                x = g[0] * rescale_ratio
                y = g[1] * rescale_ratio
                green.append((x, y))
            red = []
            for r in red_points:
                x = r[0] * rescale_ratio
                y = r[1] * rescale_ratio
                red.append((x, y))

            ## Predict Downsampled Image
            score = 0.
            time_part2 = 0.
            if green and red:
                input_point = np.concatenate((green, red))
                input_label = np.concatenate(([1] * len(green), [0] * len(red)))

                time_part2i = time.time()
                masks, scores, logits = predictor.predict(
                    point_coords=input_point,
                    point_labels=input_label,
                    multimask_output=True,
                )
                time_part2 = time.time() - time_part2i

                ## Upsample the mask
                maskDown = masks[0]
                mask = RescaleMask(maskDown, imshape, mode)

                ## Score calculation
                intersection = (mask & label).sum()
                union = (mask | label).sum()
                if intersection == 0:
                    score = 0
                else:
                    score = intersection / union

            ws['A' + str(idx + 2)] = c+1
            ws['B' + str(idx + 2)] = score
            ws['C' + str(idx + 2)] = time_part1
            ws['D' + str(idx + 2)] = time_part2
            ws['E' + str(idx + 2)] = mode
            c += 1
            idx += 1
            
    wb.save(os.path.join(dataset_name, dataset_name + '_st' + str(student) + '_mode_up.xlsx'))

Ratio Student:  1
Ratio Student:  2
Ratio Student:  3
